<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install catboost
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 15.1 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os

import polars as pl

import xgboost as xgb
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error


import lightgbm as lgb #LightGBM
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor,Pool

# Configuration

In [15]:
class config:
    METRIC = 'RMSE' #RMSE or SMAPE
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/AMP"
    
    SAMPLE_SUBMISSION = common_path + "/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = common_path + "/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = common_path + "/train_clinical_data.csv"
    TRAIN_PEPTIDES = common_path + "/train_peptides.csv"
    TRAIN_PROTEINS = common_path + "/train_proteins.csv"
    TEST_CLINICAL_DATA = common_path + "/example_test_files/test.csv"
    TEST_PEPTIDES = common_path + "/example_test_files/test_peptides.csv"
    TEST_PROTEINS = common_path + "/example_test_files/test_proteins.csv"
    #train output
    TRAIN = common_path + "/train_all.csv"
    TEST = common_path + "/test_all.csv"
    TE_UPDRS_1 = common_path + "/train_te_updrs_1.csv"
    TE_UPDRS_2 = common_path + "/train_te_updrs_2.csv"
    TE_UPDRS_3 = common_path + "/train_te_updrs_3.csv"
    TE_UPDRS_4 = common_path + "/train_te_updrs_4.csv"
    OUTPUT = common_path

# Function Definition

## make_statistic

In [4]:
#統計量の作成
def make_statistic(df,group_col:str,agg_list:list):
    df_ret = df.groupby(group_col).agg([
        pl.mean(agg_list).suffix('_mean')
        ,pl.std(agg_list).suffix("_std")
        ,pl.median(agg_list).suffix("_med")
        #,pl.col(agg_list).max().suffix("_max") 
        #,pl.col(agg_list).min().suffix("_min") 
        #,pl.quantile(agg_list',0.25).suffix("_quan1")
        #,pl.quantile(agg_list',0.75).suffix("_quan3")
        ])
    return df_ret

## display_feature_importance

In [5]:
def display_feature_importance(feature_list,model):

    # 特徴量重要度を保管する dataframe を用意
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = feature_list
    feature_importance['importance'] = model.feature_importances_

    print('****************************') 
    display(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:20])

    # order = list(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False).index)[:30]
    # # 可視化
    # plt.figure(figsize=(10, 10))
    # sns.barplot(x="importance",y="feature",data=feature_importance,order=order)
    # plt.title('importance')
    # plt.tight_layout()
    # plt.show()

## score_cal

In [6]:
def score_cal(y_true, y_pred,metric = config.METRIC):
  if metric == 'SMAPE':
    smap = np.zeros(len(y_true))
    num = np.abs((y_true + 1) - (y_pred+1))
    dem = ((np.abs(y_true + 1) + np.abs(y_pred + 1)) / 2)
    pos_ind = ((y_true+1)!=0)|((y_pred+1)!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    ret = 100 * np.mean(smap)
  if metric == 'RMSE':
    ret = np.sqrt(mean_squared_error(y_true,y_pred))

  return ret

## fit_catboost_fold1

In [7]:
def fit_catboost_fold1(X,y):

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

  train_pool = Pool(X_train, y_train)
  model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
  model.fit(train_pool,plot=False, verbose=False)  

  test_score = score_cal(y_val, model.predict(X_val))
  print('****************************')
  print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)

  return model

# Load Data

## train

In [8]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

train_clinical.replace({'On':1,'Off':0},inplace=True)
supplemental_clinical.replace({'On':1,'Off':0},inplace=True)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


## test

In [9]:
test_clinical = pd.read_csv(paths.TEST_CLINICAL_DATA)
test_peptides = pd.read_csv(paths.TEST_PEPTIDES)
test_proteins = pd.read_csv(paths.TEST_PROTEINS)
sample_submission= pd.read_csv(paths.SAMPLE_SUBMISSION)
print('Proteins shape:',test_proteins.shape)
print('Peptides shape:',test_peptides.shape)
print('Clinical shape:',test_clinical.shape)

Proteins shape: (453, 6)
Peptides shape: (2057, 7)
Clinical shape: (16, 6)


#Common

## make train_sort

In [10]:
#sort用にキー列のみ抽出
train_sort = train_clinical[['visit_id','patient_id','visit_month']]

In [11]:
test_peptides['visit_id'].unique()

array(['50423_0', '3342_6'], dtype=object)

# target encoding

In [12]:
train_clinical_te = train_clinical.copy()
train_peptides_te = train_peptides.copy()
train_proteins_te = train_proteins.copy()
supplemental_te = supplemental_clinical.copy()

## pep,uni <- updrs1-4

In [157]:
#与えられたdfを用いてvisit_idに集約
def pep_te_agg(df_key,df_peptide):
  pep_te_agg = pl.DataFrame(df_peptide[['visit_id','Peptide']]).join(train_pep_te_agg_,on='Peptide',how='left').drop('Peptide').groupby(['visit_id']).agg(
      [
          pl.mean("*").suffix("_mean_by_id"),
          pl.std("*").suffix("_std_by_id"),
      ]
  ).to_pandas()
  pep_te_agg_fill = df_key.merge(pep_te_agg,on='visit_id',how='left').groupby('patient_id').fillna(method='ffill')

  return pep_te_agg_fill

def pro_te_agg(df_key,df_protein):
  uni_te_agg = pl.DataFrame(df_protein[['visit_id','UniProt']]).join(train_uni_te_agg_,on='UniProt',how='left').drop('UniProt').groupby(['visit_id']).agg(
      [
          pl.mean("*").suffix("_mean_by_id"),
          pl.std("*").suffix("_std_by_id"),
      ]
  ).to_pandas()
  pro_te_agg_fill = df_key.merge(uni_te_agg,on='visit_id',how='left').groupby('patient_id').fillna(method='ffill')
  
  return pro_te_agg_fill

UniProt,updrs_1_mean_by_Uni,updrs_2_mean_by_Uni,updrs_3_mean_by_Uni,updrs_4_mean_by_Uni,upd23b_clinical_state_on_medication_mean_by_Uni,updrs_1_std_by_Uni,updrs_2_std_by_Uni,updrs_3_std_by_Uni,updrs_4_std_by_Uni,upd23b_clinical_state_on_medication_std_by_Uni,updrs_1_max_by_Uni,updrs_2_max_by_Uni,updrs_3_max_by_Uni,updrs_4_max_by_Uni,upd23b_clinical_state_on_medication_max_by_Uni
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""P43652""",6.565666,5.821764,17.3125,1.902998,0.504444,5.334088,5.933761,15.02809,3.074362,0.500537,33.0,29.0,78.0,20.0,1.0
"""P17174""",6.43956,5.674326,16.931452,1.881905,0.492718,5.293826,5.898799,14.979614,3.094158,0.500555,33.0,29.0,78.0,20.0,1.0
"""P02766""",6.566479,5.821161,17.300567,1.908612,0.506637,5.330702,5.931414,15.016371,3.077419,0.50051,33.0,29.0,78.0,20.0,1.0
"""P31997""",6.335624,5.608247,16.505202,1.997763,0.494083,5.324414,6.00134,15.024757,3.226501,0.500706,33.0,29.0,78.0,20.0,1.0
"""Q08380""",6.568885,5.825679,17.306528,1.911972,0.505543,5.332622,5.932356,15.022227,3.079086,0.500524,33.0,29.0,78.0,20.0,1.0
"""P04196""",6.568885,5.825679,17.306528,1.911972,0.505543,5.332622,5.932356,15.022227,3.079086,0.500524,33.0,29.0,78.0,20.0,1.0
"""O00391""",6.152174,5.4375,16.009642,2.005181,0.520408,5.276176,5.839244,14.795849,3.322489,0.500435,28.0,29.0,78.0,20.0,1.0
"""P25311""",6.568885,5.825679,17.306528,1.911972,0.505543,5.332622,5.932356,15.022227,3.079086,0.500524,33.0,29.0,78.0,20.0,1.0
"""P02790""",6.566479,5.821161,17.300567,1.908612,0.506637,5.330702,5.931414,15.016371,3.077419,0.50051,33.0,29.0,78.0,20.0,1.0


In [151]:
#欠損値処理
#train_clinical_te.dropna(subset=['updrs_1'],inplace=True)
#train_clinical_te.dropna(subset=['updrs_2'],inplace=True)
#train_clinical_te.dropna(subset=['updrs_3'],inplace=True) 

train_pep_te = train_peptides_te[['visit_id','Peptide']].merge(train_clinical_te[['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']]
                                                ,on='visit_id',how='inner')
train_uni_te = train_proteins_te[['visit_id','UniProt']].merge(train_clinical_te[['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']]
                                                ,on='visit_id',how='inner')

#集計処理 by Pep/Uni
train_pep_te_agg_ = pl.DataFrame(train_pep_te.drop(['visit_id'],axis=1)).groupby(['Peptide']).agg(
    [
        pl.mean("*").suffix("_mean_by_Pep"),
        pl.std("*").suffix("_std_by_Pep"),
        pl.median("*").suffix("_median_by_Pep"),
        pl.max("*").suffix("_max_by_Pep"),
        #pl.min("*").suffix("_min_by_Pep"),
    ]
)

train_uni_te_agg_ = pl.DataFrame(train_uni_te.drop(['visit_id'],axis=1)).groupby(['UniProt']).agg(
    [
        pl.mean("*").suffix("_mean_by_Uni"),
        pl.std("*").suffix("_std_by_Uni"),
        pl.max("*").suffix("_max_by_Uni"),
        #pl.min("*").suffix("_min_by_Uni"),
    ]
)

train_pep_te_agg_fill = pep_te_agg(train_sort,train_peptides_te)
train_pro_te_agg_fill = pro_te_agg(train_sort,train_proteins_te)



,visit_id,visit_month,updrs_1_mean_by_Pep_mean_by_id,updrs_2_mean_by_Pep_mean_by_id,updrs_3_mean_by_Pep_mean_by_id,updrs_4_mean_by_Pep_mean_by_id,upd23b_clinical_state_on_medication_mean_by_Pep_mean_by_id,updrs_1_std_by_Pep_mean_by_id,updrs_2_std_by_Pep_mean_by_id,updrs_3_std_by_Pep_mean_by_id,...,updrs_1_median_by_Pep_std_by_id,updrs_2_median_by_Pep_std_by_id,updrs_3_median_by_Pep_std_by_id,updrs_4_median_by_Pep_std_by_id,upd23b_clinical_state_on_medication_median_by_Pep_std_by_id,updrs_1_max_by_Pep_std_by_id,updrs_2_max_by_Pep_std_by_id,updrs_3_max_by_Pep_std_by_id,updrs_4_max_by_Pep_std_by_id,upd23b_clinical_state_on_medication_max_by_Pep_std_by_id
0,55_0,0,6.499588,5.775430,17.190754,1.886132,0.496048,5.308492,5.918159,15.022675,...,0.309644,0.148859,0.426175,0.0,0.491248,1.317100,0.212778,3.240491,1.951132,0.0
1,55_3,3,6.499588,5.775430,17.190754,1.886132,0.496048,5.308492,5.918159,15.022675,...,0.309644,0.148859,0.426175,0.0,0.491248,1.317100,0.212778,3.240491,1.951132,0.0
2,55_6,6,6.501296,5.777437,17.195011,1.886238,0.496024,5.309869,5.920009,15.025634,...,0.308946,0.144019,0.415266,0.0,0.490972,1.291932,0.212234,3.135952,1.944267,0.0
3,55_9,9,6.501296,5.777437,17.195011,1.886238,0.496024,5.309869,5.920009,15.025634,...,0.308946,0.144019,0.415266,0.0,0.490972,1.291932,0.212234,3.135952,1.944267,0.0
4,55_12,12,6.498244,5.774333,17.187653,1.886255,0.496214,5.308340,5.918437,15.022644,...,0.307975,0.150702,0.450148,0.0,0.491258,1.343520,0.215993,3.263424,1.952518,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610,65043_48,48,6.501298,5.780114,17.200105,1.885122,0.495667,5.309976,5.922890,15.027763,...,0.311186,0.155418,0.445967,0.0,0.490867,1.264117,0.197362,3.153475,1.963058,0.0
2611,65043_54,54,6.501298,5.780114,17.200105,1.885122,0.495667,5.309976,5.922890,15.027763,...,0.311186,0.155418,0.445967,0.0,0.490867,1.264117,0.197362,3.153475,1.963058,0.0
2612,65043_60,60,6.501298,5.780114,17.200105,1.885122,0.495667,5.309976,5.922890,15.027763,...,0.311186,0.155418,0.445967,0.0,0.490867,1.264117,0.197362,3.153475,1.963058,0.0
2613,65043_72,72,6.501298,5.780114,17.200105,1.885122,0.495667,5.309976,5.922890,15.027763,...,0.311186,0.155418,0.445967,0.0,0.490867,1.264117,0.197362,3.153475,1.963058,0.0


## updrs <- updrs

In [32]:
train_clinical_te
supplemental_te

train_clisup_ = pd.concat([train_clinical_te,supplemental_te])
train_clisup = train_clisup_[['updrs_1','updrs_2','updrs_3','updrs_4']]

for i in range(4) :
  train_clisup_tmp = pl.DataFrame(train_clisup).groupby(['updrs_{0}'.format(i+1)]).agg(
    [
        pl.mean("*").suffix("_mean_by_"+'updrs_{0}'.format(i+1)),
        #pl.std("*").suffix("_std_by_Pep"),
        pl.median("*").suffix("_median_by_"+'updrs_{0}'.format(i+1)),
        #pl.max("*").suffix("_max_by_Pep"),
        pl.min("*").suffix("_min_by_"+'updrs_{0}'.format(i+1)),
    ]
  ).to_pandas()
  locals()["te_updrs_{0}".format(i+1)] = train_clisup_tmp

te_updrs_1.to_csv(paths.OUTPUT+'/train_te_updrs_1.csv',index=False)
te_updrs_2.to_csv(paths.OUTPUT+'/train_te_updrs_2.csv',index=False)
te_updrs_3.to_csv(paths.OUTPUT+'/train_te_updrs_3.csv',index=False)
te_updrs_4.to_csv(paths.OUTPUT+'/train_te_updrs_4.csv',index=False)

## month <- updrs1-4

In [96]:
train_clisup = train_clisup_[['visit_month','updrs_1','updrs_2','updrs_3','updrs_4']]

train_te_month_agg = pl.DataFrame(train_clisup).groupby(['visit_month']).agg(
    [
        pl.mean("*").suffix("_mean_by_month"),
        pl.std("*").suffix("_std_by_month"),
        pl.median("*").suffix("_median_by_month"),
        #pl.min("*").suffix("_min_by_Uni"),
    ]
).to_pandas()

,visit_month,updrs_1_mean_by_month,updrs_2_mean_by_month,updrs_3_mean_by_month,updrs_4_mean_by_month,updrs_1_std_by_month,updrs_2_std_by_month,updrs_3_std_by_month,updrs_4_std_by_month,updrs_1_median_by_month,updrs_2_median_by_month,updrs_3_median_by_month,updrs_4_median_by_month
0,72,8.365591,8.817204,26.450549,2.289157,5.400855,6.109412,15.279081,2.666550,7.0,8.0,29.0,1.0
1,30,6.739899,7.043038,22.650633,1.260736,4.955552,5.539949,12.320981,2.516920,6.0,6.0,22.0,0.0
2,0,5.725108,5.734848,18.312684,1.186517,4.452248,5.161743,12.375385,2.404725,5.0,4.0,18.0,0.0
3,42,8.339869,8.372549,22.720779,1.868613,5.694138,6.672688,14.128494,2.977409,7.0,7.0,23.0,0.0
4,108,9.416667,8.166667,25.250000,2.571429,8.836477,9.093787,21.629630,2.699206,5.5,6.5,34.5,2.0
5,6,5.938636,6.670455,22.143836,1.427419,4.956150,5.233765,11.905418,3.118717,5.0,6.0,21.0,0.0
6,84,7.730000,8.230000,22.589474,2.938462,5.644082,7.788693,18.387956,2.760644,6.5,8.0,21.0,3.0
7,24,6.304348,6.153318,19.889145,1.290850,4.952648,5.431122,14.085115,2.462249,5.0,5.0,19.0,0.0
8,96,7.706897,7.775862,21.267857,4.233333,5.556752,8.117650,21.044102,2.990598,7.0,6.0,19.5,4.0
9,18,6.344353,6.776860,21.576177,0.938697,4.819377,5.325568,12.560758,2.129481,5.0,6.0,21.0,0.0


#Protain

In [193]:
def feature_proteins(df_key,df_proteins):
  #UniProtのcount情報を付与
  #pro_Uniprot_count = pd.DataFrame(df_proteins['UniProt'].value_counts()).reset_index()
  #df_proteins = df_proteins.merge(pro_Uniprot_count.rename(columns={'UniProt':'value_counts'}),left_on='UniProt',right_on='index',how='left')

  #統計量の作成
  df_proteins_ = pl.DataFrame(df_proteins).select([
      pl.mean('NPX').over('visit_id').alias('NPX_mean_by_id')
    ,pl.std('NPX').over('visit_id').alias('NPX_std_by_id')
    ,pl.median('NPX').over('visit_id').alias('NPX_median_by_id')
    ,pl.mean('NPX').over('UniProt').alias('NPX_mean_by_Uni')
    ,pl.std('NPX').over('UniProt').alias('NPX_std_by_Uni')
    ,pl.median('NPX').over('UniProt').alias('NPX_median_by_Uni')
    #,pl.mean('value_counts').over('visit_id').alias('Unicount_mean_by_id')
    #,pl.std('value_counts').over('visit_id').alias('Unicount_std_by_id')
    #,pl.median('value_counts').over('visit_id').alias('Unicount_median_by_id')
  ])

  df_proteins_agg_ = df_proteins[['visit_id']].join(df_proteins_.to_pandas())
  df_proteins_agg = df_key.merge(df_proteins_agg_.groupby('visit_id').mean().reset_index(),on='visit_id',how='left').sort_values(['patient_id','visit_month']).reset_index(drop=True).drop(['visit_month'],axis=1)

  #欠損値を1つ前の値で埋める
  df_proteins_agg_fill = df_proteins_agg.groupby('patient_id').fillna(method='ffill')
  #df_proteins_toall = df_proteins_agg_fill.merge(train_pro_te_agg_fill,on='visit_id',how='left').copy()
  df_proteins_toall = df_proteins_agg_fill.merge(pro_te_agg(df_key,df_proteins),on='visit_id',how='left').copy()

  return df_proteins_toall

train_proteins_feature = feature_proteins(train_sort,train_proteins)

#Peptide

In [194]:
def feature_peptide(df_key,df_peptides):
  #pep_Peptide_count = pd.DataFrame(df_peptides['Peptide'].value_counts()).reset_index()
  #df_peptides = train_peptides.merge(pep_Peptide_count.rename(columns={'Peptide':'value_counts'}),left_on='Peptide',right_on='index',how='left')

  df_peptides_ = pl.DataFrame(df_peptides).select([
      pl.mean('PeptideAbundance').over('visit_id').alias('Abu_mean_by_id')
    ,pl.std('PeptideAbundance').over('visit_id').alias('Abu_std_by_id')
    ,pl.median('PeptideAbundance').over('visit_id').alias('Abu_median_by_id')
    ,pl.mean('PeptideAbundance').over('Peptide').alias('Abu_mean_by_Pep')
    ,pl.std('PeptideAbundance').over('Peptide').alias('Abu_std_by_Pep')
    ,pl.median('PeptideAbundance').over('Peptide').alias('Abu_median_by_Pep')
    #,pl.mean('value_counts').over('visit_id').alias('Pepcount_mean_by_id')
    #,pl.std('value_counts').over('visit_id').alias('Pepcount_std_by_id')
    #,pl.median('value_counts').over('visit_id').alias('Pepcount_median_by_id')
  ])

  df_peptides_agg_ = df_peptides[['visit_id']].join(df_peptides_.to_pandas())
  df_peptides_agg_  = df_key.merge(df_peptides_agg_.groupby('visit_id').mean().reset_index(),on='visit_id',how='left').sort_values(['patient_id','visit_month']).reset_index(drop=True).drop(['visit_month'],axis=1)

  #欠損値を1つ前の値で埋める
  df_peptides_agg_fill = df_peptides_agg_.groupby('patient_id').fillna(method='ffill')
  df_peptides_agg_toall = df_peptides_agg_fill.merge(pep_te_agg(df_key,df_peptides),on='visit_id',how='left').copy()
  return df_peptides_agg_toall

train_peptides_feature = feature_peptide(train_sort,train_peptides)

#Clinical

In [20]:
#train_clinical['visit_month'].nunique()
#train_clinical['visit_month'].unique()
#train_clinical['visit_month'].value_counts()
#[  0,   3,   6,   9,  12,  18,  24,  30,  36,  42,  48,  54,  60, 72,  84,  96, 108] 17types
#⇒ Phase_1 初期 [0,3,6,9,12,18,24] Phase_2 中期 [30,36,42,48,54,60] Phase_3 後期[72,84,96,108]

In [21]:
#欠損値処理
train_clinical.dropna(subset=['updrs_1'],inplace=True)
train_clinical.dropna(subset=['updrs_2'],inplace=True)
train_clinical.dropna(subset=['updrs_3'],inplace=True) #とりあえず

#upd23dの置き換え
#train_clinical.replace({'On':1,'Off':2},inplace=True)

train_clinical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2588 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2588 non-null   object 
 1   patient_id                           2588 non-null   int64  
 2   visit_month                          2588 non-null   int64  
 3   updrs_1                              2588 non-null   float64
 4   updrs_2                              2588 non-null   float64
 5   updrs_3                              2588 non-null   float64
 6   updrs_4                              1562 non-null   float64
 7   upd23b_clinical_state_on_medication  1280 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 182.0+ KB


In [22]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

train_clinical_rolling = train_clinical.copy()

for i in range(3):
  train_clinical_rolling['updrs_{0}_rolling'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].rolling(3,min_periods=1).mean().reset_index(drop=True)
  train_clinical_rolling['updrs_{0}_ewm'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].transform(lambda x: x.ewm(halflife=10).mean()).reset_index(drop=True)
train_clinical_rolling.head()


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,updrs_1_rolling,updrs_1_ewm,updrs_2_rolling,updrs_2_ewm,updrs_3_rolling,updrs_3_ewm
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,10.000000,10.000000,6.000000,6.000000,15.000000,15.000000
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,10.000000,10.000000,6.500000,6.517322,20.000000,20.173217
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,9.333333,9.286627,7.666667,7.759546,24.666667,25.105042
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,8.666667,8.930796,8.666667,8.102607,29.666667,26.458798
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,8.666667,9.175259,9.666667,8.536426,35.000000,29.783493


# supplemental_clinical

In [23]:
supplemental_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,1.0
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


#Train All

In [ ]:
def make_df_all(df_cli,df_pro,df_pep):
  df_propep = df_pro.merge(df_pep,on='visit_id',how='left')
  df_all = df_cli.merge(df_propep,on='visit_id',how='left')
  return df_all

In [197]:
train_clinical_toall = train_clinical.merge(train_te_month_agg,on='visit_month',how='left')

#train_clinical_toall.shape,train_proteins_feature.shape,train_peptides_feature.shape

train_all = make_df_all(train_clinical_toall,train_proteins_feature,train_peptides_feature)
print('shape',train_all.shape)
train_all.head()

train_all.describe()

train_all.to_csv(paths.OUTPUT+'/train_all.csv',index=False)

train_all.columns
train_proteins_feature.columns
train_peptides_feature.columns


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,updrs_1_mean_by_month,updrs_2_mean_by_month,...,updrs_1_median_by_Pep_std_by_id,updrs_2_median_by_Pep_std_by_id,updrs_3_median_by_Pep_std_by_id,updrs_4_median_by_Pep_std_by_id,upd23b_clinical_state_on_medication_median_by_Pep_std_by_id,updrs_1_max_by_Pep_std_by_id,updrs_2_max_by_Pep_std_by_id,updrs_3_max_by_Pep_std_by_id,updrs_4_max_by_Pep_std_by_id,upd23b_clinical_state_on_medication_max_by_Pep_std_by_id
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,5.725108,5.734848,...,0.309644,0.148859,0.426175,0.0,0.491248,1.317100,0.212778,3.240491,1.951132,0.0
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,5.513043,6.634783,...,0.309644,0.148859,0.426175,0.0,0.491248,1.317100,0.212778,3.240491,1.951132,0.0
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,5.938636,6.670455,...,0.308946,0.144019,0.415266,0.0,0.490972,1.291932,0.212234,3.135952,1.944267,0.0
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,6.080808,7.204082,...,0.308946,0.144019,0.415266,0.0,0.490972,1.291932,0.212234,3.135952,1.944267,0.0
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,5.736402,5.765690,...,0.307975,0.150702,0.450148,0.0,0.491258,1.343520,0.215993,3.263424,1.952518,0.0


#Test All

3342と50423の6month刻みの予測を行う -> 必要分のみサブ
3342...3342_0は無視。3342_6を3342_0として扱う。50423はそのまま

In [198]:
#clinical
test_clinical.drop(test_clinical.index[test_clinical.visit_id == '3342_6'], inplace=True)
test_clinical.drop(test_clinical.index[test_clinical.visit_id == '50423_6'], inplace=True)
#peptide
test_peptides['visit_id'].replace('3342_6', '3342_0', inplace=True)
test_peptides['visit_month'].replace(6, 0, inplace=True)
#protein
test_proteins['visit_id'].replace('3342_6', '3342_0', inplace=True)
test_proteins['visit_month'].replace(6, 0, inplace=True)


In [200]:
test_sort = test_clinical[['visit_id','visit_month','patient_id']]
test_sort.drop_duplicates(inplace=True)

test_train_peptides = pd.concat([train_peptides,test_peptides.drop(['group_key'],axis=1)],axis=0).reset_index(drop=True)
test_peptide_feature = feature_peptide(test_sort,test_train_peptides)

test_train_proteins = pd.concat([train_proteins,test_proteins.drop(['group_key'],axis=1)],axis=0).reset_index(drop=True)
test_protein_feature = feature_proteins(test_sort,test_train_proteins)

test_clinical_toall = test_sort.merge(train_te_month_agg,on='visit_month',how='left')

#test_all = make_df_all(test_clinical_toall,test_protein_feature,test_peptide_feature)
test_propep = test_protein_feature.merge(test_peptide_feature,on='visit_id',how='left')
test_all = test_clinical_toall.merge(test_propep,on='visit_id',how='left')


#test_all.drop(['updrs_test','row_id','group_key'],axis=1,inplace=True)
test_all.drop_duplicates(inplace=True)

print(test_all.shape)
#test_clinical_toall.shape
#test_protein_feature.shape
#test_peptide_feature.shape

test_all.to_csv(paths.OUTPUT+'/test_all.csv',index=False)


<ipython-input-200-7b8b4d51318f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sort.drop_duplicates(inplace=True)


(2, 99)


# Fit Start


In [201]:
df_train = train_all.copy()
df_test = test_all.copy()

# Make Test_Val Data

In [28]:
#pivot
df_train_pv=df_train[['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4']].dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
df_train_pv.head()

#testデータ用の候補探し
test_patient_candidate_list = list(df_train_pv[
    ~df_train_pv[0].isna() & ~df_train_pv[6].isna() & ~df_train_pv[12].isna() & ~df_train_pv[24].isna()
]['patient_id'].values)
len(test_patient_candidate_list)


56

In [29]:
#test_patient_list = [1517,3863] #決め打ち　Top1,2
test_patient_list = [4923,7265] #決め打ち  Top3,4

test_month_list = [0,6,12,24]   #TESTと同一
#test_month_list = [0,3,6,9,12,18,24]
df_test_val = df_train[df_train['patient_id'].isin(test_patient_list) & df_train['visit_month'].isin(test_month_list)].reset_index(drop=True)

df_test_val_true = pd.DataFrame()

for i in range(0,3):
  df_test_val_true_ = pd.DataFrame()
  df_test_val_true_['id']=df_test_val[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_true_['updrs']=df_test_val[['updrs_{0}'.format(i+1)]]
  df_test_val_true = pd.concat([df_test_val_true,df_test_val_true_],axis=0)

# Fit

## fit main

In [202]:

#predict_order_list = [2,1,3]
predict_order_list = [1,2,3]
#predict_order_list = [3,1,2]
#predict_order_list = [1]

df_train_ = df_train.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  
  train_X = df_train_.values
  train_y = df_train['updrs_{0}'.format(order)].values
  model = fit_catboost_fold1(train_X,train_y)

  if order == 1:
        model_1 = model
        df_te = te_updrs_1
  if order == 2:
        model_2 = model
        df_te = te_updrs_2
  if order == 3:
        model_3 = model
        df_te = te_updrs_3

  #display importance
  display_feature_importance(df_train_.columns,model)

  #予測値をtrainにセット
  df_train_['updrs_{0}'.format(order)] = model.predict(train_X)
  df_train_['updrs_{0}_round'.format(order)] = df_train_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_train_ = df_train_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_train_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)
  print('train shape:',df_train_.shape)


****************************
Test data average score : 3.7156  seed  100
****************************


feature
updrs_1_std_by_Pep_mean_by_id                                  5.950356
updrs_2_std_by_Pep_mean_by_id                                  4.069412
updrs_1_median_by_Pep_std_by_id                                3.479150
updrs_1_median_by_Pep_mean_by_id                               2.628452
updrs_2_max_by_Pep_mean_by_id                                  2.421334
updrs_2_median_by_Pep_std_by_id                                2.409764
Abu_median_by_id                                               2.026531
updrs_1_max_by_Pep_std_by_id                                   2.003564
updrs_4_max_by_Pep_std_by_id                                   1.994635
updrs_1_max_by_Uni_std_by_id                                   1.960829
updrs_4_max_by_Uni_std_by_id                                   1.925439
updrs_1_max_by_Uni_mean_by_id                                  1.873434
updrs_1_mean_by_Pep_mean_by_id                                 1.616220
updrs_2_max_by_Uni_mean_by_id                           

train shape: (2588, 107)
****************************
Test data average score : 3.7886  seed  100
****************************


feature
updrs_1                                                       10.010614
updrs_3_mean_by_updrs_1                                        4.163433
updrs_2_std_by_Pep_mean_by_id                                  3.378742
updrs_2_mean_by_updrs_1                                        2.944793
updrs_3_std_by_Pep_mean_by_id                                  2.895400
updrs_2_median_by_updrs_1                                      2.868160
updrs_3_median_by_updrs_1                                      2.478600
NPX_median_by_id                                               2.420435
updrs_2_std_by_Uni_mean_by_id                                  2.115327
updrs_4_mean_by_updrs_1                                        1.982686
updrs_2_median_by_Pep_std_by_id                                1.914727
upd23b_clinical_state_on_medication_mean_by_Uni_mean_by_id     1.888742
updrs_3_std_by_Uni_mean_by_id                                  1.575750
updrs_2_max_by_Uni_mean_by_id                           

train shape: (2588, 117)
****************************
Test data average score : 9.5150  seed  100
****************************


feature
updrs_2                                                        13.965420
updrs_1_mean_by_updrs_2                                         5.395815
updrs_3_mean_by_updrs_2                                         4.522386
updrs_1_median_by_updrs_2                                       3.810024
updrs_3_median_by_updrs_2                                       3.802154
updrs_3_std_by_Pep_mean_by_id                                   2.722465
updrs_4_mean_by_updrs_2                                         2.495148
updrs_3_std_by_Uni_mean_by_id                                   1.995329
updrs_1                                                         1.813549
NPX_median_by_id                                                1.716904
updrs_2_median_by_Pep_std_by_id                                 1.634839
Abu_median_by_id                                                1.629126
updrs_4_max_by_Pep_std_by_id                                    1.561189
upd23b_clinical_state_on_medication_mean_by

train shape: (2588, 127)


## predict Test_Val

In [203]:
df_test_val_ = df_test_val.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  test_X = df_test_val_.values
  test_y = df_test_val['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te_val = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te_val = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te_val = te_updrs_3

  df_test_val_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_val_['updrs_{0}_round'.format(order)] = df_test_val_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te_val.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_val_ = df_test_val_.merge(df_te_val,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_val_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_val_pred_ = df_test_val[['visit_id']].join(df_test_val_[['updrs_1','updrs_2','updrs_3']])

df_test_val_pred = pd.DataFrame()
for i in range(0,3):
  df_test_val_pred_vs = pd.DataFrame()
  df_test_val_pred_vs['id'] = df_test_val_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_pred_vs['updrs']=df_test_val_pred_[['updrs_{0}'.format(i+1)]]
  df_test_val_pred = pd.concat([df_test_val_pred,df_test_val_pred_vs],axis=0)

print('****************************')
print('Score Predict : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values))
print('Score Predict SMAPE : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values,'SMAPE'))

****************************
Score Predict :  9.615585009145715
Score Predict SMAPE :  34.54174079965486


## display Test_Val True vs Pred

In [204]:
#どれぐらい差があるか調べる
df_test_val_truepred = df_test_val_true.merge(df_test_val_pred,on='id',how='left').rename(columns={'updrs_x':'true','updrs_y':'pred'}).round(1)
df_test_val_truepred['diff'] = abs(df_test_val_truepred['true']-df_test_val_truepred['pred'])
df_test_val_truepred['diff_ratio'] = round((df_test_val_truepred['diff'] / df_test_val_truepred['true']) * 100,0)
display(df_test_val_truepred)

,id,true,pred,diff,diff_ratio
0,4923_0_updrs_1,2.0,6.8,4.8,240.0
1,4923_6_updrs_1,6.0,10.1,4.1,68.0
2,4923_12_updrs_1,6.0,5.9,0.1,2.0
3,4923_24_updrs_1,4.0,5.5,1.5,38.0
4,7265_0_updrs_1,10.0,6.8,3.2,32.0
5,7265_6_updrs_1,12.0,10.1,1.9,16.0
6,7265_12_updrs_1,12.0,13.6,1.6,13.0
7,7265_24_updrs_1,15.0,14.6,0.4,3.0
8,4923_0_updrs_2,3.0,6.4,3.4,113.0
9,4923_6_updrs_2,3.0,7.4,4.4,147.0


## predict Test for colab

In [205]:
df_test_ = df_test.drop(['visit_id'],axis=1)

num_features_used = sum(model_1.get_feature_importance() > 0)
print("Number of features used:", num_features_used)

for order in predict_order_list:
  test_X = df_test_.values
  #test_y = df_test['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te = te_updrs_3

  df_test_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_['updrs_{0}_round'.format(order)] = df_test_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_ = df_test_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_pred_ = df_test[['visit_id']].reset_index(drop=True).join(df_test_[['updrs_1','updrs_2','updrs_3']]).round(2)

df_test_pred = pd.DataFrame()
for i in range(0,3):
  df_test_pred_vs = pd.DataFrame()
  df_test_pred_vs['id'] = df_test_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_pred_vs['updrs']=df_test_pred_[['updrs_{0}'.format(i+1)]]
  df_test_pred = pd.concat([df_test_pred,df_test_pred_vs],axis=0)

display(df_test_pred)

Number of features used: 92


,id,updrs
0,3342_0_updrs_1,7.72
1,50423_0_updrs_1,7.70
0,3342_0_updrs_2,6.18
1,50423_0_updrs_2,6.20
0,3342_0_updrs_3,14.69
1,50423_0_updrs_3,15.20


# Inference for kaggle notebook

## import Library

In [36]:
if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  import amp_pd_peptide
  env = amp_pd_peptide.make_env()
  iter_test = env.iter_test()

In [37]:
# def map_test(x):
#     updrs = x.split('_')[2] + '_' + x.split('_')[3]
#     month = int(x.split('_plus_')[1].split('_')[0])
#     visit_id = x.split('_')[0] + '_' + x.split('_')[1]
#     if updrs=='updrs_4':
#         rating = 0
#     elif updrs =='updrs_1':
#         rating = df[df.visit_id == visit_id]['pred1'].values[0]
#     elif updrs =='updrs_2':
#         rating = df[df.visit_id == visit_id]['pred2'].values[0]
#     else: #updrs =='updrs_3'
#         rating = df[df.visit_id == visit_id]['pred3'].values[0]
#     return rating

if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  counter = 0

  for (test, test_peptides, test_proteins, sample_submission) in iter_test:

    #1.common 特徴量の作成

    #2-1.predict to updrs_1

    #2-2.updrs_1を使った追加の特徴量作成

    #3-1.predict to updrs_2

    #3-2.updrs_1を使った追加の特徴量作成

    #4.predict to updrs_3
    
    #5.submissionの作成

    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)
    env.predict(sample_submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

In [38]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

# df_log = pd.DataFrame() #Log用DFの定義
# models = []                          # 各 fold のモデル
# scores = 0.0   
# best_score = np.inf
# preds = np.zeros(len(X_test))

# gf = GroupShuffleSplit(n_splits=config.N_FOLD,test_size=0.2,random_state=config.RANDOM_STATE)
# groups = X_train[:,0]

# for fold, (trn_idx, val_idx) in enumerate(gf.split(X_train,y_train,groups)):

#     X_trn, y_trn = X_train[trn_idx,:], y_train[trn_idx]
#     X_val, y_val = X_train[val_idx,:], y_train[val_idx]

#     train_pool = Pool(X_trn, y_trn)
#     validate_pool = Pool(X_val, y_val)

#     model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
#     model.fit(train_pool, eval_set = validate_pool, early_stopping_rounds = 10, use_best_model=True, plot=False, verbose=False)  
    
#     score = score_cal(y_val,model.predict(X_val))
#     scores += score / config.N_FOLD
#     models.append(model)

#     if score < best_score:
#         best_score = score
#         best_model = model
#         best_fold = fold+1
#     print("Fold :" , fold+1)

# print(f"fold average score : {scores:.4f}")
# print(f"fold best score : {best_score:.4f}"," FOLD ",best_fold)
# #****************************
# # test データ

# # 各モデルで推論
# for model in models:
#     preds += model.predict(X_test) / len(models) 
# test_score = score_cal(y_test, preds)
# pred_best = best_model.predict(X_test)

# print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)
# print(f"Test data best socre : {score_cal(y_test, pred_best):.4f}"," seed ", config.RANDOM_STATE)
# print("***********************")
# #display_feature_importance(train_column_list,best_model)
# feature_importance = pd.DataFrame()
# feature_importance['feature'] = train_column_list
# feature_importance['importance'] = best_model.feature_importances_

# feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:30]